In [4]:
# Import necessary libraries
import numpy as np
import time
import psutil
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
import faiss

# Utility to measure memory usage
def get_memory():
    process = psutil.Process()
    mem = process.memory_info().rss / 1024 / 1024  # MB
    return mem

# Generate synthetic dense embeddings for testing
np.random.seed(42)
n_refs = 10000  # number of reference vectors
dim = 768       # embedding dimension
n_queries = 5

reference_embeddings = np.random.rand(n_refs, dim).astype('float32')
query_embeddings = np.random.rand(n_queries, dim).astype('float32')

# Normalize embeddings for cosine similarity (dot product on L2-normalized vectors)
faiss.normalize_L2(reference_embeddings)
faiss.normalize_L2(query_embeddings)

# --- Sparse Embedding preparation ---
def sparsify_embeddings(embeddings, sparsity_threshold=0.1):
    # Zero out small values and convert to sparse CSR
    sparse_data = np.where(np.abs(embeddings) < sparsity_threshold, 0, embeddings)
    return csr_matrix(sparse_data)

# Sparsify reference and query embeddings
sparsity_threshold = 0.1
ref_sparse = sparsify_embeddings(reference_embeddings, sparsity_threshold)
query_sparse = sparsify_embeddings(query_embeddings, sparsity_threshold)

# --- Benchmark dense cosine similarity ---
start_mem = get_memory()
start = time.time()
dense_sim = cosine_similarity(query_embeddings, reference_embeddings)
dense_time = time.time() - start
dense_mem = get_memory() - start_mem

print(f"Dense similarity took {dense_time:.4f}s, Memory: {dense_mem:.2f} MB")

# --- Benchmark sparse cosine similarity ---
start_mem = get_memory()
start = time.time()
sparse_sim = cosine_similarity(query_sparse, ref_sparse)
sparse_time = time.time() - start
sparse_mem = get_memory() - start_mem

print(f"Sparse similarity took {sparse_time:.4f}s, Memory: {sparse_mem:.2f} MB")

# --- Hierarchical Matcher with Faiss + clustering ---
class HierarchicalSparseMatcher:
    def __init__(self, reference_embeddings, n_clusters=100, sparsity_threshold=0.1):
        self.reference_embeddings = reference_embeddings
        self.sparsity_threshold = sparsity_threshold

        # Sparsify embeddings
        sparse_data = np.where(np.abs(reference_embeddings) < sparsity_threshold, 0, reference_embeddings)
        self.ref_sparse = csr_matrix(sparse_data)

        # Cluster dense embeddings for coarse search
        self.kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=1000)
        self.cluster_labels = self.kmeans.fit_predict(reference_embeddings)

        # Build fine-grained Faiss indices per cluster
        self.cluster_indices = {}
        for i in range(n_clusters):
            cluster_mask = self.cluster_labels == i
            if np.sum(cluster_mask) > 0:
                cluster_embeds = reference_embeddings[cluster_mask]
                faiss.normalize_L2(cluster_embeds)
                index = faiss.IndexFlatIP(cluster_embeds.shape[1])
                index.add(cluster_embeds.astype('float32'))
                self.cluster_indices[i] = {
                    'index': index,
                    'mask': cluster_mask,
                    'embeddings': cluster_embeds
                }

    def search(self, query_embeddings, k=5, search_clusters=3):
        # Sparsify queries similarly
        query_data = np.where(np.abs(query_embeddings) < self.sparsity_threshold, 0, query_embeddings)
        query_sparse = csr_matrix(query_data)

        results = []
        for idx, query in enumerate(query_sparse):
            query_dense = query_embeddings[idx]

            # Coarse cluster search by closest cluster centroids (dense)
            cluster_distances = np.linalg.norm(self.kmeans.cluster_centers_ - query_dense.reshape(1, -1), axis=1)
            closest_clusters = np.argsort(cluster_distances)[:search_clusters]

            all_similarities, all_indices = [], []

            # Fine search with Faiss inside clusters
            for cluster_id in closest_clusters:
                if cluster_id in self.cluster_indices:
                    cluster_info = self.cluster_indices[cluster_id]
                    sims, idxs = cluster_info['index'].search(query_dense.reshape(1, -1).astype('float32'), k)
                    global_idxs = np.where(cluster_info['mask'])[0][idxs[0]]
                    all_similarities.extend(sims[0])
                    all_indices.extend(global_idxs)

            if all_similarities:
                top_k_idx = np.argsort(all_similarities)[-k:][::-1]
                results.append({
                    'similarities': [all_similarities[i] for i in top_k_idx],
                    'indices': [all_indices[i] for i in top_k_idx]
                })

        return results

# Instantiate matcher
matcher = HierarchicalSparseMatcher(reference_embeddings, n_clusters=50, sparsity_threshold=0.1)

# Benchmark hierarchical search
start_mem = get_memory()
start = time.time()
results = matcher.search(query_embeddings, k=5, search_clusters=3)
hier_time = time.time() - start
hier_mem = get_memory() - start_mem

print(f"Hierarchical search time: {hier_time:.4f}s, Memory: {hier_mem:.2f} MB")
print("Sample results for first query:")
print(results[0])


Dense similarity took 0.0347s, Memory: 1.20 MB
Sparse similarity took 0.0101s, Memory: 0.35 MB
Hierarchical search time: 0.0059s, Memory: 0.00 MB
Sample results for first query:
{'similarities': [np.float32(0.7783653), np.float32(0.77661383), np.float32(0.7745984), np.float32(0.7744216), np.float32(0.77437496)], 'indices': [np.int64(1286), np.int64(7688), np.int64(8635), np.int64(9916), np.int64(1795)]}





- **Dense search**: Brute force over all embeddings; simple but slow on large datasets.
- **Sparse search**: Reduces computation by thresholding small values to zero and using sparse matrix multiplication; faster but requires sparse_dot_topn package.
- **Hierarchical search**: Uses clustering to limit search to a few clusters, combined with Faiss for accurate similarity within clusters; balances speed and accuracy well.

This benchmarking highlights how hierarchical clustering-based approximate nearest neighbor search can offer significant speedups and memory efficiency on large embedding datasets while maintaining good accuracy.


# Sparse Embeddings and Hierarchical Search Optimization

This notebook demonstrates an approach to optimize similarity search for high-dimensional embeddings using sparsification and hierarchical clustering.

## 1. Sparsification of Embeddings

We convert dense embeddings into sparse representations by zeroing out small magnitude values below a threshold (`sparsity_threshold`). This reduces memory and compute cost when calculating similarity.

The sparse embeddings are stored in CSR (Compressed Sparse Row) format for efficient operations.

## 2. Similarity Search

- **Dense similarity:** We use `cosine_similarity` on dense embeddings as a baseline.
- **Sparse similarity:** We compute cosine similarity on the sparse CSR matrices using scikit-learn's optimized implementation.

## 3. Hierarchical Search with Clustering and Faiss

To speed up large-scale search, we apply a two-level hierarchical search:

- **Coarse search:** Cluster the reference embeddings with MiniBatchKMeans. For each query, select the closest clusters based on centroid distances.
- **Fine search:** Within those clusters, run an approximate top-k similarity search using Faiss on dense embeddings.

## 4. Benchmark Results

We benchmark:

- Dense cosine similarity
- Sparse cosine similarity
- Hierarchical search combining clustering and Faiss

This approach balances accuracy and efficiency, enabling scalable similarity search for large embedding datasets.

